In [ ]:
import sys
import os
import pandas as pd
import datetime

import typesense
from pprint import pprint
# first_name,last_name,middle_name,charges,case_date,filing_date


In [ ]:
from src.services.cases import get_cases

In [ ]:
sys.path.append("..")
os.environ["ROOT_PATH"] = "/Users/yassine-elazrak/Fubloo.com/app"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/yassine-elazrak/Fubloo.com/app/configuration/fubloo-app-1f213ca274de.json"


In [ ]:
cases_found = get_cases(
    start_date=datetime.datetime(2024, 1, 1),
    end_date=datetime.datetime(2024, 1, 2)
)

In [ ]:
cases_found

In [ ]:
def concat_dict_values(dict_list):
    if dict_list is None:
        return ""
    
    concatenated_string = ""
    for dictionary in dict_list:
        for value in dictionary.values():
            concatenated_string += str(value) + " " 
    return concatenated_string.strip() 

In [ ]:

client = typesense.Client({
  'api_key': 'Cw3AX4YbY!?QK7z',
  'nodes': [{
    'host': 'app.fubloo.com',
    'port': '443',
    'path': '/typesense',
    'protocol': 'https'
  }],
  'connection_timeout_seconds': 2
})

In [ ]:
schema = {
  'name': 'cases',
  'fields': [
    {'name': 'case_id', 'type': 'string'},
    {'name': 'formatted_party_address', 'type': 'string'},
    {'name': 'formatted_party_name', 'type': 'string'},
    {'name': 'court_id', 'type': 'string'},
    {'name': 'description', 'type': 'string'},
    {'name': 'first_name', 'type': 'string'},
    {'name': 'last_name', 'type': 'string', 'facet': True},
    {'name': 'middle_name', 'type': 'string'},
    {'name': 'charges', 'type': 'string'},
    {'name': 'case_date', 'type': 'string', 'facet': True},
    {'name': 'filing_date', 'type': 'string', 'facet': True},
    {'name': 'case_date_timestamp', 'type': 'int32'}
  ],
  'default_sorting_field': 'case_date_timestamp'
}

In [ ]:
create_response = client.collections.create(schema)
create_response

## Create document

In [ ]:
# for case in cases_found:
#     case_json =  case.model_dump(exclude_none=True)
#     charges = concat_dict_values(case_json.get("charges",""))
#     case_date = case_json.get("case_date_timestamp")
#     if case_date is None:
#         case_date = datetime.today()
#     document = {
#         "id": case_json.get("case_id",""),
#         "case_id": case_json.get("case_id",""),
#         'formatted_party_address': case_json.get("formatted_party_address",""),
#         'formatted_party_name': case_json.get("formatted_party_name",""),
#         'court_id': case_json.get("court_id",""),
#         'description': case_json.get("description",""),
#         'first_name': case_json.get("first_name",""),
#         'last_name': case_json.get("last_name",""),
#         'middle_name': case_json.get("middle_name",""),
#         'case_date': str(case_date),
#         'filing_date': str(case_json.get("filing_date","")),
#         "charges":"",
#         'case_date_timestamp': int(case_date.timestamp()),
#     } 
    
#     client.collections['cases'].documents.create(document)

    

In [ ]:
from datetime import datetime

def get_case_document(case):
    case_json = case.model_dump(exclude_none=True)
    charges = concat_dict_values(case_json.get("charges", ""))
    
    case_date = case_json.get("case_date_timestamp")
    case_date = datetime.fromtimestamp(case_date) if case_date else datetime.today()

    return {
        "id": case_json.get("case_id", ""),
        "case_id": case_json.get("case_id", ""),
        'formatted_party_address': case_json.get("formatted_party_address", ""),
        'formatted_party_name': case_json.get("formatted_party_name", ""),
        'court_id': case_json.get("court_id", ""),
        'description': case_json.get("description", ""),
        'first_name': case_json.get("first_name", ""),
        'last_name': case_json.get("last_name", ""),
        'middle_name': case_json.get("middle_name", ""),
        'case_date': str(case_date),
        'filing_date': str(case_json.get("filing_date", "")),
        "charges": charges,
        'case_date_timestamp': int(case_date.timestamp()),
    }


In [ ]:
for case in cases_found:
    case_id = case.model_dump(exclude_none=True).get("case_id", "")
    
    # Check if a document with this case_id already exists
    existing_doc = client.collections['cases'].documents[case_id].retrieve()
    if existing_doc:
        print(f"Document with ID {case_id} already exists, skipping.")
        continue

    # If it doesn't exist, create a new document
    document = get_case_document(case)
    client.collections['cases'].documents.create(document)

In [ ]:
# client.collections['cases'].documents.retrieve(case_id)
print(client.collections['cases'].documents.get('24TRs10000001')) #.retrieve()


## Searchbale search parameters

In [ ]:
search_parameters = {
  'q'         : 'le',
  'query_by'  : 'last_name ',
  'sort_by'   : 'case_date_timestamp:desc'
}

In [ ]:
res = client.collections['cases'].documents.search(search_parameters)

print(res)

In [ ]:
pprint(client.collections.retrieve())


In [ ]:
#delete colections 
client.collections["cases"].delete()


In [ ]:
charges = [{'charge_offense': {'description': 'INOPERABLE TAIL LIGHT', 'jurisdiction': 'CHICAGO POLICE DEPT', 'degree': 'Traffic', 'statute': 'MCC-9-76-050CC'}, 'filed_date': '01/01/2024', 'filing_agency_description': 'Chicago Police Department', 'citation_number': 'TX011287', 'current_charge_num': '1', 'case_party_id': 0, 'amended_date': None, 'charge_id': 57777786, 'offense_date': '12/31/2023', 'party_id': 76769652}, {'charge_offense': {'description': 'DRIVING ON SUSPENDED LICENSE', 'jurisdiction': 'CHICAGO POLICE DEPT', 'degree': 'Class A Misdemeanor', 'statute': '625 ILCS 5/6-303(a)'}, 'filed_date': '01/01/2024', 'filing_agency_description': 'Chicago Police Department', 'citation_number': 'TX011288', 'current_charge_num': '2', 'case_party_id': 0, 'amended_date': None, 'charge_id': 57777787, 'offense_date': '12/31/2023', 'party_id': 76769652}, {'charge_offense': {'description': 'OPERATE UNINSURED MTR VEHICLE', 'jurisdiction': 'CHICAGO POLICE DEPT', 'degree': 'Petty Offense', 'statute': '625 ILCS 5/3-707(a)'}, 'filed_date': '01/01/2024', 'filing_agency_description': 'Chicago Police Department', 'citation_number': 'TX001289', 'current_charge_num': '3', 'case_party_id': 0, 'amended_date': None, 'charge_id': 57777791, 'offense_date': '12/31/2023', 'party_id': 76769652}]


In [ ]:
concat_dict_values(charges)

In [ ]:
client.collections['casess'].retrieve()